In [1]:
import pandas as pd
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import os
from sklearn.model_selection import train_test_split

In [2]:
# Hinged Square Loss
class SquaredHingeLoss(nn.Module):
    def __init__(self, margin=1):
        super(SquaredHingeLoss, self).__init__()
        self.margin = margin

    def forward(self, predicted, y):
        low, high = y[:, 0:1], y[:, 1:2]
        loss_low = torch.relu(low - predicted + self.margin)
        loss_high = torch.relu(predicted - high + self.margin)
        loss = loss_low + loss_high
        return torch.mean(torch.square(loss))

In [3]:
class LinearModel(nn.Module):
    def __init__(self, input_size):
        super(LinearModel, self).__init__()
        self.fc1 = nn.Linear(input_size, 1)

    def forward(self, x):
        x = self.fc1(x)
        return x

In [4]:
dataset = 'systematic'
chosen_feature = ['log_variance', 'loglog_count']

In [5]:
# Load data
folds_df = pd.read_csv(f'../../training_data/{dataset}/folds.csv')
features_df = pd.read_csv(f'../../training_data/{dataset}/features.csv')[['sequenceID'] + chosen_feature]
target_df = pd.read_csv(f'../../training_data/{dataset}/target.csv')
for test_fold in range(1, 7):
    # Split data into training and test sets
    train_ids = folds_df[folds_df['fold'] != test_fold]['sequenceID']
    test_ids = folds_df[folds_df['fold'] == test_fold]['sequenceID']

    features_df_train = features_df[features_df['sequenceID'].isin(train_ids)]
    features_df_test = features_df[features_df['sequenceID'].isin(test_ids)]
    target_df_train = target_df[target_df['sequenceID'].isin(train_ids)]
    target_df_test = target_df[target_df['sequenceID'].isin(test_ids)]

    # Create X_train, y_train, X_test, y_test
    X_train = features_df_train[chosen_feature].to_numpy()
    X_test = features_df_test[chosen_feature].to_numpy()
    y_train = target_df_train.iloc[:, 1:].to_numpy()
    y_test = target_df_test.iloc[:, 1:].to_numpy()


    # Convert data to PyTorch tensors
    X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
    y_train_tensor = torch.tensor(y_train, dtype=torch.float32)
    X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
    y_test_tensor = torch.tensor(y_test, dtype=torch.float32)

    # Initialize the model, loss function, and optimizer
    input_dim = X_train.shape[1]
    model = LinearModel(input_dim)
    criterion = SquaredHingeLoss()
    optimizer = optim.Adam(model.parameters())

    # Training with early stopping based on validation loss
    best_loss = float('inf')
    patience = 100
    patience_counter = 0

    for epoch in range(100000):
        model.train()
        optimizer.zero_grad()
        
        # Forward pass on subtrain data
        outputs_subtrain = model(X_train_tensor)
        train_loss = criterion(outputs_subtrain, y_train_tensor)
        
        # Backward pass and optimization
        train_loss.backward()
        optimizer.step()
        
        # Early stopping check based on validation loss
        train_loss_value = train_loss.item()
        if train_loss_value < best_loss:
            best_loss = train_loss_value
            best_model = model.state_dict()  # Save the best model state
            patience_counter = 0
        else:
            patience_counter += 1
        
        # # Print train, validation, and test losses every 1000 epochs
        # if epoch % 2000 == 0:
        #     model.eval()
        #     with torch.no_grad():
        #         outputs_test = model(X_test_tensor)
        #         test_loss = criterion(outputs_test, y_test_tensor)
        #     print(f"Epoch [{epoch:5d}] \t Train Loss: {train_loss.item():.8f} \t  Test Loss: {test_loss.item():.8f}")
        
        if patience_counter >= patience:
            # print(f"Early stopping at epoch {epoch}")
            break

    # Load the best model
    model.load_state_dict(best_model)

    # Final evaluation on the test set
    model.eval()
    with torch.no_grad():
        outputs_test = model(X_test_tensor)
        test_loss = criterion(outputs_test, y_test_tensor)

    # Print the final test loss
    print(f"Test fold: {test_fold} \t Test Loss: {test_loss.item():.8f}")

Test fold: 1 	 Test Loss: 0.08095080
Test fold: 2 	 Test Loss: 0.06858940
Test fold: 3 	 Test Loss: 0.07166144
Test fold: 4 	 Test Loss: 0.08258455
Test fold: 5 	 Test Loss: 0.07257746
Test fold: 6 	 Test Loss: 0.08724432
